## 1. Imports and Configuration

In [ ]:
# Core libraries
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Audio processing
import librosa
import librosa.display

# Machine learning
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
SAMPLE_RATE = 22050  # Standard sample rate
DURATION = 5.0       # Fixed audio duration in seconds
N_MFCC = 13          # Number of MFCC coefficients

# Paths
DATA_DIR = 'data/raw'
OUTPUT_DIR = 'output'
CLASSES = ['car', 'tram']

print("Libraries loaded successfully!")
print(f"Sample Rate: {SAMPLE_RATE} Hz")
print(f"Audio Duration: {DURATION} seconds")
print(f"Classes: {CLASSES}")

## 2. Data Loading

Load audio files from the `data/raw/car` and `data/raw/tram` folders.

In [ ]:
def load_audio_files(data_dir, classes, sr=SAMPLE_RATE):
    """
    Load all audio files from class folders.
    
    Returns:
        audio_data: list of (audio_array, sample_rate, filename)
        labels: list of class labels
    """
    audio_data = []
    labels = []
    
    for class_name in classes:
        class_dir = os.path.join(data_dir, class_name)
        
        if not os.path.exists(class_dir):
            print(f"Warning: Directory {class_dir} does not exist!")
            continue
        
        files = [f for f in os.listdir(class_dir) 
                 if f.endswith(('.wav', '.mp3', '.flac', '.ogg', '.m4a'))]
        
        print(f"Loading {len(files)} files from '{class_name}' folder...")
        
        for filename in files:
            filepath = os.path.join(class_dir, filename)
            try:
                # Load audio file
                audio, _ = librosa.load(filepath, sr=sr)
                audio_data.append((audio, sr, filename))
                labels.append(class_name)
            except Exception as e:
                print(f"Error loading {filename}: {e}")
    
    return audio_data, labels

# Load the data
audio_data, labels = load_audio_files(DATA_DIR, CLASSES)

print(f"\nTotal samples loaded: {len(audio_data)}")
print(f"Class distribution:")
for cls in CLASSES:
    count = labels.count(cls)
    print(f"  - {cls}: {count} samples")

## 3. Normalization

Apply audio normalization:
1. **Peak normalization**: Scale audio to [-1, 1] range
2. **Pad/Truncate**: Ensure all audio clips are exactly 5 seconds

In [ ]:
def normalize_audio(audio):
    """
    Apply peak normalization to audio signal.
    Scales the audio to have maximum amplitude of 1.0
    """
    max_val = np.max(np.abs(audio))
    if max_val > 0:
        return audio / max_val
    return audio

def pad_or_truncate(audio, sr, duration=DURATION):
    """
    Pad with zeros or truncate audio to fixed duration.
    """
    target_length = int(sr * duration)
    
    if len(audio) > target_length:
        # Truncate to target length
        return audio[:target_length]
    elif len(audio) < target_length:
        # Pad with zeros
        padding = target_length - len(audio)
        return np.pad(audio, (0, padding), mode='constant')
    return audio

def preprocess_audio(audio_data):
    """
    Apply normalization and padding to all audio samples.
    """
    processed_audio = []
    
    for audio, sr, filename in audio_data:
        # Step 1: Peak normalization
        audio_norm = normalize_audio(audio)
        
        # Step 2: Pad or truncate to fixed duration
        audio_fixed = pad_or_truncate(audio_norm, sr)
        
        processed_audio.append((audio_fixed, sr, filename))
    
    return processed_audio

# Preprocess all audio
processed_audio = preprocess_audio(audio_data)

print(f"Preprocessing complete!")
print(f"All audio clips normalized and padded/truncated to {DURATION} seconds")
print(f"Sample length: {len(processed_audio[0][0])} samples ({DURATION}s at {SAMPLE_RATE}Hz)")

## 4. Feature Extraction

Extract audio features for classification:
- **MFCCs** (13 coefficients): Captures timbral characteristics
- **Spectral Centroid**: "Brightness" of the sound
- **Zero Crossing Rate (ZCR)**: Noisiness indicator
- **RMS Energy**: Volume/loudness
- **Spectral Rolloff**: Frequency distribution

For each feature, we compute **mean** and **standard deviation** across time frames.

In [ ]:
def extract_features(audio, sr, n_mfcc=N_MFCC):
    """
    Extract audio features from a single audio sample.
    
    Returns:
        feature_vector: numpy array of aggregated features
    """
    features = []
    
    # 1. MFCCs (13 coefficients x 2 stats = 26 features)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    features.extend(np.mean(mfccs, axis=1))  # Mean of each MFCC
    features.extend(np.std(mfccs, axis=1))   # Std of each MFCC
    
    # 2. Spectral Centroid (2 features)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
    features.append(np.mean(spectral_centroid))
    features.append(np.std(spectral_centroid))
    
    # 3. Zero Crossing Rate (2 features)
    zcr = librosa.feature.zero_crossing_rate(audio)[0]
    features.append(np.mean(zcr))
    features.append(np.std(zcr))
    
    # 4. RMS Energy (2 features)
    rms = librosa.feature.rms(y=audio)[0]
    features.append(np.mean(rms))
    features.append(np.std(rms))
    
    # 5. Spectral Rolloff (2 features)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
    features.append(np.mean(spectral_rolloff))
    features.append(np.std(spectral_rolloff))
    
    return np.array(features)

def extract_all_features(processed_audio):
    """
    Extract features from all audio samples.
    """
    feature_list = []
    
    for i, (audio, sr, filename) in enumerate(processed_audio):
        features = extract_features(audio, sr)
        feature_list.append(features)
        
        if (i + 1) % 10 == 0:
            print(f"Processed {i + 1}/{len(processed_audio)} files...")
    
    return np.array(feature_list)

# Extract features
print("Extracting features...")
X = extract_all_features(processed_audio)

# Create feature names for reference
feature_names = []
for i in range(N_MFCC):
    feature_names.append(f'mfcc_{i+1}_mean')
for i in range(N_MFCC):
    feature_names.append(f'mfcc_{i+1}_std')
feature_names.extend(['spectral_centroid_mean', 'spectral_centroid_std',
                      'zcr_mean', 'zcr_std',
                      'rms_mean', 'rms_std',
                      'spectral_rolloff_mean', 'spectral_rolloff_std'])

print(f"\nFeature extraction complete!")
print(f"Feature matrix shape: {X.shape}")
print(f"Number of features per sample: {len(feature_names)}")

## 5. Data Split and Scaling

- Split data into training (80%) and testing (20%) sets
- Apply StandardScaler to normalize features (important for SVM!)

In [ ]:
# Convert labels to numeric values
label_mapping = {'car': 0, 'tram': 1}
y = np.array([label_mapping[label] for label in labels])

print(f"Labels encoded: {label_mapping}")
print(f"Label distribution: car={np.sum(y==0)}, tram={np.sum(y==1)}")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Maintain class balance
)

print(f"\nData split:")
print(f"  Training set: {X_train.shape[0]} samples")
print(f"  Testing set: {X_test.shape[0]} samples")

# Feature scaling using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Use same scaler fitted on training data

print(f"\nFeatures scaled using StandardScaler")
print(f"  Training mean (should be ~0): {X_train_scaled.mean():.6f}")
print(f"  Training std (should be ~1): {X_train_scaled.std():.6f}")

## 6. SVM Training

Train a Support Vector Machine classifier with RBF kernel.
We use GridSearchCV to find optimal hyperparameters (C and gamma).

In [ ]:
# Define SVM with RBF kernel
svm_model = SVC(kernel='rbf', random_state=42)

# Hyperparameter grid for tuning
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1]
}

print("Training SVM with GridSearchCV for hyperparameter tuning...")
print(f"Parameter grid: {param_grid}")

# Grid search with cross-validation
grid_search = GridSearchCV(
    svm_model, 
    param_grid, 
    cv=5,  # 5-fold cross-validation
    scoring='accuracy',
    verbose=1,
    n_jobs=-1  # Use all CPU cores
)

# Fit the model
grid_search.fit(X_train_scaled, y_train)

# Best model
best_model = grid_search.best_estimator_

print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")

## 7. Evaluation

Evaluate the trained model on the test set using:
- **Accuracy**: Overall correctness
- **Precision**: How many predicted positives are actually positive
- **Recall**: How many actual positives were correctly identified
- **F1-Score**: Harmonic mean of precision and recall
- **Confusion Matrix**: Detailed breakdown of predictions

In [ ]:
# Make predictions on test set
y_pred = best_model.predict(X_test_scaled)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("="*50)
print("MODEL EVALUATION RESULTS")
print("="*50)
print(f"\nAccuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")

# Detailed classification report
print("\n" + "="*50)
print("CLASSIFICATION REPORT")
print("="*50)
print(classification_report(y_test, y_pred, target_names=CLASSES))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

## 8. Visualizations

Create visualizations for the report:
1. Sample waveforms from each class
2. Sample spectrograms
3. Confusion matrix heatmap
4. Feature importance analysis

In [ ]:
# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=CLASSES, yticklabels=CLASSES)
plt.title('Confusion Matrix - SVM Classifier')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'confusion_matrix.png'), dpi=150)
plt.show()

print(f"Confusion matrix saved to {OUTPUT_DIR}/confusion_matrix.png")

In [ ]:
# Plot sample waveforms from each class
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

for idx, class_name in enumerate(CLASSES):
    # Find first sample of this class
    class_idx = labels.index(class_name)
    audio, sr, filename = processed_audio[class_idx]
    
    # Waveform
    axes[idx, 0].plot(np.linspace(0, DURATION, len(audio)), audio, color='blue', linewidth=0.5)
    axes[idx, 0].set_title(f'{class_name.upper()} - Waveform')
    axes[idx, 0].set_xlabel('Time (s)')
    axes[idx, 0].set_ylabel('Amplitude')
    axes[idx, 0].set_ylim(-1, 1)
    
    # Spectrogram
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio)), ref=np.max)
    librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='hz', ax=axes[idx, 1])
    axes[idx, 1].set_title(f'{class_name.upper()} - Spectrogram')
    axes[idx, 1].set_xlabel('Time (s)')
    axes[idx, 1].set_ylabel('Frequency (Hz)')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'waveforms_spectrograms.png'), dpi=150)
plt.show()

print(f"Waveforms and spectrograms saved to {OUTPUT_DIR}/waveforms_spectrograms.png")

In [ ]:
# Feature statistics comparison between classes
df_features = pd.DataFrame(X, columns=feature_names)
df_features['class'] = labels

# Plot mean feature values per class for key features
key_features = ['mfcc_1_mean', 'mfcc_2_mean', 'spectral_centroid_mean', 
                'zcr_mean', 'rms_mean', 'spectral_rolloff_mean']

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for i, feature in enumerate(key_features):
    df_features.boxplot(column=feature, by='class', ax=axes[i])
    axes[i].set_title(feature)
    axes[i].set_xlabel('Class')
    axes[i].set_ylabel('Value')

plt.suptitle('Feature Distribution by Class', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'feature_distribution.png'), dpi=150)
plt.show()

print(f"Feature distribution plot saved to {OUTPUT_DIR}/feature_distribution.png")

In [ ]:
# Summary statistics
print("="*60)
print("PROJECT SUMMARY")
print("="*60)
print(f"\nDataset:")
print(f"  - Total samples: {len(audio_data)}")
print(f"  - Car samples: {labels.count('car')}")
print(f"  - Tram samples: {labels.count('tram')}")
print(f"  - Audio duration: {DURATION} seconds")
print(f"  - Sample rate: {SAMPLE_RATE} Hz")

print(f"\nFeatures:")
print(f"  - Total features: {len(feature_names)}")
print(f"  - MFCCs: {N_MFCC} coefficients (mean + std)")
print(f"  - Spectral features: centroid, ZCR, RMS, rolloff")

print(f"\nModel:")
print(f"  - Classifier: Support Vector Machine (SVM)")
print(f"  - Kernel: RBF")
print(f"  - Best C: {grid_search.best_params_['C']}")
print(f"  - Best gamma: {grid_search.best_params_['gamma']}")

print(f"\nResults:")
print(f"  - Accuracy: {accuracy*100:.2f}%")
print(f"  - Precision: {precision*100:.2f}%")
print(f"  - Recall: {recall*100:.2f}%")
print(f"  - F1-Score: {f1*100:.2f}%")

print("\n" + "="*60)
print("All results saved to the 'output' folder.")
print("="*60)